In [9]:
import numpy as np


class Aligner:

    def __init__(self, seq1, seq2, gapPenalty, missPenalty, matchScore):
        self.seq1 = seq1
        self.seq2 = seq2
        self.gapPenalty = gapPenalty
        self.missPenalty = missPenalty
        self.matchScore = matchScore
        self.alignMatrix = np.zeros(
            (len(self.seq1)+1, len(self.seq2)+1), dtype=int)
        self.traceBackMatrix = np.zeros(
            (len(self.seq1)+1, len(self.seq2)+1), dtype='U4')
        self.indexToTrace = {
            0: "d",
            1: "l",
            2: "u"
        }
        self.finalScore = 0
        self.identity = 0

        
    def getValue(self, i, j):
        if self.seq1[i-1] == self.seq2[j-1]:
            missOrMatch = self.matchScore
        else:
            missOrMatch = self.missPenalty

        possibleValues = [
            self.alignMatrix[i-1][j-1] + missOrMatch,
            self.alignMatrix[i][j-1] + self.gapPenalty,
            self.alignMatrix[i-1][j] + self.gapPenalty
        ]

        return max(possibleValues), possibleValues.index(max(possibleValues))


    def align(self):

        for row in self.traceBackMatrix:
            row[0] = "u"

        for i in range(len(self.traceBackMatrix[0])):
            self.traceBackMatrix[0][i] = "l"
        self.traceBackMatrix[0][0] = "f"

        accGap = 0
        for row in self.alignMatrix:
            row[0] = accGap
            accGap += self.gapPenalty

        accGap = 0
        for i in range(len(self.alignMatrix[0])):
            self.alignMatrix[0][i] = accGap
            accGap += self.gapPenalty

        for i, j in np.ndindex(self.alignMatrix.shape):
            if i == 0:
                continue
            if j == 0:
                continue

            self.alignMatrix[i][j], index = self.getValue(i, j)
            self.traceBackMatrix[i][j] = self.indexToTrace[index]

        self.finalScore = self.alignMatrix[len(self.seq1)][len(self.seq2)]
        self.makeAlignment()

        
    def makeAlignment(self):

        s1 = ''
        s2 = ''

        i = len(self.seq1)
        j = len(self.seq2)
        alignType = self.traceBackMatrix[i][j]

        while alignType != 'f':

            if alignType == 'd':
                s1 = self.seq1[i-1] + s1
                s2 = self.seq2[j-1] + s2
                i -= 1
                j -= 1

            if alignType == 'l':
                s1 = '-' + s1
                s2 = self.seq2[j-1] + s2
                j -= 1

            if alignType == 'u':
                s1 = self.seq1[i-1] + s1
                s2 = '-' + s2
                i -= 1

            alignType = self.traceBackMatrix[i][j]

        self.s1 = s1
        self.s2 = s2

        self.getIdentity()


    def getIdentity(self):

        ident = 0

        for i in range(len(self.s1)):
            if self.s1[i] == self.s2[i]:
                ident += 1

        totalPositions = max([len(self.seq1), len(self.seq2)])
        self.identity = ident/totalPositions

    def printResults(self):
        print(self.alignMatrix, '\n')
        print(self.s1)
        print(self.s2, '\n')
        print('Final Score:', self.finalScore)
        print('Identity:', self.identity)

In [10]:
aligner = Aligner("AATCG","AACG", -2, -1, 1)
aligner.align()
aligner.printResults()

[[  0  -2  -4  -6  -8]
 [ -2   1  -1  -3  -5]
 [ -4  -1   2   0  -2]
 [ -6  -3   0   1  -1]
 [ -8  -5  -2   1   0]
 [-10  -7  -4  -1   2]] 

AATCG
AA-CG 

Final Score: 2
Identity: 0.8


In [3]:
file1 = open("hemoglobins/human.txt","r")
human = file1.read()
file1.close()
print(human)

VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKY


In [6]:
animalList = ["chicken", "cow", "deer", "horse", "pig", "trout", "wolf"]
animals = {}

for animal in animalList:
    file1 = open("hemoglobins/"+animal+".txt","r")
    animalSequence = file1.read()
    file1.close()
    animals[animal] = animalSequence
    
    
for x, y in animals.items():
    print(x, y,"\n")


chicken MLTAEDKKLIQQAWEKAASHQEEFGAEALTRMFTTYPQTKTYFPHFDLSPGSDQVRGHGKKVLGALGNAVKNVDNLSQAMAELSNLHAYNLRVDPVNFKLLSQCIQVVLAVHMGKDYTPEVHAAFDKFLSAVSAVLAEKYR 

cow VLSAADKGNVKAAWGKVGGHAAEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVNFKLLSHSLLVTLASHLPSDFTPAVHASLDKFLANVSTVLTSKYR 

deer VLSAANKSNVKAAWGKVGGNAPAYGAQALQRMFLSFPTTKTYFPHFDLSHGSAQQKAHGQKVANALTKAQGHLNDLPGTLSNLSNLHAHKLRVNPVNFKLLSHSLLVTLASHLPTNFTPAVHANLNKFLANDSTVLTSKYR 

horse VLSAADKTNVKAAWSKVGGHAGEYGAEALERMFLGFPTTKTYFPHFDLSHGSAQVKAHGKKVGDALTLAVGHLDDLPGALSNLSDLHAHKLRVDPVNFKLLSHCLLSTLAVHLPNDFTPAVHASLDKFLSSVSTVLTSKYR 

pig VLSAADKANVKAAWGKVGGQAGAHGAEALERMFLGFPTTKTYFPHFNLSHGSDQVKAHGQKVADALTKAVGHLDDLPGALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHHPDDFNPSVHASLDKFLANVSTVLTSKYR 

trout XSLTAKDKSVVKAFWGKISGKADVVGAEALGRMLTAYPQTKTYFSHWADLSPGSGPVKKHGGIIMGAIGKAVGLMDDLVGGMSALSDLHAFKLRVDPGNFKILSHNILVTLAIHFPSDFTPEVHIAVDKFLAAVSAALADKYR 

wolf VLSPADKTNIKSTWDKIGGHAGDYGGEALDRTFQSFPTTKTYFPHFDLSPGSAQVKAHGKKVADALTTAVAHLDDLPGALSALSDLHAYKLRVDPV

In [7]:
a = animals["cow"]
print(a)

VLSAADKGNVKAAWGKVGGHAAEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGAKVAAALTKAVEHLDDLPGALSELSDLHAHKLRVDPVNFKLLSHSLLVTLASHLPSDFTPAVHASLDKFLANVSTVLTSKYR


In [14]:
file1 = open("hemoglobins/human.txt", "r")
human = file1.read()
file1.close()

animals = {}

animalList = ["chicken", "cow", "deer", "horse", "pig", "trout", "wolf"]
animals = {}

for animal in animalList:
    file1 = open("hemoglobins/"+animal+".txt", "r")
    animalSequence = file1.read()
    file1.close()
    animals[animal] = animalSequence


gap = -4
match = 5
missmatch = -3

scores = {}


for animal in animalList:

    aligner = Aligner(human, animals[animal], gap, missmatch, match)
    aligner.align()
    scores["human vs "+animal] = (aligner.finalScore, aligner.identity)

    print("Results for human vs "+animal+":\n")
    aligner.printResults()
    print("\n")

Results for human vs chicken:

[[   0   -4   -8 ... -556 -560 -564]
 [  -4   -3   -7 ... -547 -551 -555]
 [  -8   -7    2 ... -538 -542 -546]
 ...
 [-552 -543 -534 ...  243  239  235]
 [-556 -547 -538 ...  252  248  244]
 [-560 -551 -542 ...  248  257  253]] 

VLSPA-DKTNVKAAWGKVGA-HAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALS---A-LSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLAS-VSTVLTSKY-
ML-TAEDKKLIQQAWEK-AASHQEEFGAEALTRMFTTYPQTKTYFPHFDLSPGSDQVRGHGKKVLGALGNAVKNV-D--N-LSQAMAELSNLHAYNLRVDPVNFKLLSQCIQVVLAVHMGKDYTPEVHAAFDKFL-SAVSAVLAEKYR 

Final Score: 253
Identity: 0.6312056737588653


Results for human vs cow:

[[   0   -4   -8 ... -556 -560 -564]
 [  -4    5    1 ... -547 -551 -555]
 [  -8    1   10 ... -538 -542 -546]
 ...
 [-552 -543 -534 ...  550  546  542]
 [-556 -547 -538 ...  559  555  551]
 [-560 -551 -542 ...  555  564  560]] 

VLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVST

In [24]:
bestVal = (0, 0)
bestKey = ''
draws = []

for key, value in scores.items():
    
    if value[0] > bestVal[0]:
        bestVal = value
        bestKey = key
        draws = []
    
    elif value[0] == bestVal[0]:
        if value[1] > bestVal[1]:
            bestVal = value
            bestKey = key
            draws = []
            
        elif value[1] == bestVal[1]:
            draws.append(key)

In [25]:
print(bestVal, bestKey)

(560, 0.8723404255319149) human vs cow


In [26]:
print(draws)

['human vs horse']


In [27]:
print("\nThe best score was achieved when "+bestKey +
          " were compared. \nObtained score:", scores[bestKey][0], "\nObtained identity:", scores[bestKey][1])


The best score was achieved when human vs cow were compared. 
Obtained score: 560 
Obtained identity: 0.8723404255319149


In [28]:
print("\nMore than one sequence aligment produced the same final score and identity.\n")
print(bestKey,"- (score, identity):", scores[bestKey])

for d in draws:
    print(d,"- (score, identity):", scores[d])


More than one sequence aligment produced the same final score and identity.

human vs cow - (score, identity): (560, 0.8723404255319149)
human vs horse - (score, identity): (560, 0.8723404255319149)
